In [1]:
import numpy as np
import pandas as pd
import psycopg2

In [2]:
import json

with open('config.json') as f:
    conf = json.load(f)
    host = conf['host']
    database = conf['database']
    user = conf['user']
    passw = conf['passw']

In [3]:
conn_str = "host={} dbname={} user={} password={}".format(host, database, user, passw)
conn = psycopg2.connect(conn_str)

In [4]:
query = '''
SELECT      DISTINCT inc.incident_id AS INCIDENT_ID,
            vic.victim_id AS VICTIM_ID,
            date_part('year',inc.incident_date),
            vic.victim_type_id AS VICTIM_TYPE_ID,
            ty.victim_type_name AS VICTIM_TYPE,
            vic.age_num,
            vic.age_range_low_num AS AGE_RANGE_LOW,
            vic.age_range_high_num AS AGE_RANGE_HIGH,
            vic.age_id,
            age.age_code,
            age.age_name,
            vic.sex_code AS VICTIM_SEX,
            oft.crime_against AS CRIME_AGAINST,
            oft.offense_name AS OFFENSE,
            oft.offense_category_name AS OFFENSE_CATEGORY,
            oft.offense_group AS OFFENSE_GROUP,
            off.location_id AS LOCATION_ID,
            loc.location_name AS LOCATION_NAME,
            ori.fips AS FIPS,
            ori.countyname AS COUNTY,
            ori.name AS ORI_NAME,
            ags.population AS POPULATION,
            ags.population_group_desc AS POPULATION_DESCRIPTION,
            ags.total_officers AS OFFICERS,
            ags.total_civilians AS CIVILIANS
            
FROM        nibrs_victim as vic
JOIN        nibrs_victim_type as ty
ON          vic.victim_type_id = ty.victim_type_id
JOIN        nibrs_age as age
ON          age.age_id = vic.age_id
JOIN        nibrs_offense as off
ON          off.incident_id = vic.incident_id
JOIN        nibrs_offense_type as oft
ON          oft.offense_type_id = off.offense_type_id
JOIN        nibrs_location_type as loc
ON          off.location_id = loc.location_id
JOIN        nibrs_incident as inc
ON          inc.incident_id = vic.incident_id
JOIN        cde_agencies as ags
ON          ags.agency_id = inc.agency_id
JOIN        ori_to_fips as ori
ON          ori.ori9 = ags.ori

WHERE       vic.victim_type_id = 4
AND         inc.incident_date BETWEEN '2014-01-01' AND '2016-12-31';
'''

In [5]:
df = pd.read_sql(query, con=conn)
df.head(3).T

,0,1,2
incident_id,73180647,73180648,73180649
victim_id,79599709,79599710,79599711
date_part,2014,2014,2014
victim_type_id,4,4,4
victim_type,Individual,Individual,Individual
age_num,35,60,38
age_range_low,NaN,NaN,NaN
age_range_high,NaN,NaN,NaN
age_id,5,5,5
age_code,AG,AG,AG


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478607 entries, 0 to 478606
Data columns (total 25 columns):
incident_id               478607 non-null int64
victim_id                 478607 non-null int64
date_part                 478607 non-null float64
victim_type_id            478607 non-null int64
victim_type               478607 non-null object
age_num                   469263 non-null float64
age_range_low             184635 non-null float64
age_range_high            184635 non-null float64
age_id                    478607 non-null int64
age_code                  478607 non-null object
age_name                  478607 non-null object
victim_sex                478607 non-null object
crime_against             478607 non-null object
offense                   478607 non-null object
offense_category          478607 non-null object
offense_group             478607 non-null object
location_id               478607 non-null int64
location_name             478607 non-null object
fips    

### Random Forest Regressor

In [7]:
model_df = df[['age_num','victim_sex','crime_against','offense_category','location_id','population','officers','civilians','county']]

model_df = pd.get_dummies(model_df, columns = ['victim_sex','crime_against','offense_category','location_id'])
model_df.head()

,age_num,population,officers,civilians,county,victim_sex_F,victim_sex_M,victim_sex_U,crime_against_Not a Crime,crime_against_Person,...,location_id_38,location_id_39,location_id_40,location_id_41,location_id_42,location_id_43,location_id_44,location_id_45,location_id_46,location_id_47
0,35.0,71780,144,34,BELL,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,60.0,71780,144,34,BELL,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,38.0,71780,144,34,BELL,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,66.0,71780,144,34,BELL,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,57.0,71780,144,34,BELL,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478607 entries, 0 to 478606
Data columns (total 80 columns):
age_num                                                      469263 non-null float64
population                                                   478607 non-null int64
officers                                                     478607 non-null int64
civilians                                                    478607 non-null int64
county                                                       478607 non-null object
victim_sex_F                                                 478607 non-null uint8
victim_sex_M                                                 478607 non-null uint8
victim_sex_U                                                 478607 non-null uint8
crime_against_Not a Crime                                    478607 non-null uint8
crime_against_Person                                         478607 non-null uint8
crime_against_Property                                  

### Drop missing age rows (<1% of data)

In [9]:
model_df.dropna(axis=0, subset=['age_num'],inplace=True)

In [10]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 469263 entries, 0 to 478606
Data columns (total 80 columns):
age_num                                                      469263 non-null float64
population                                                   469263 non-null int64
officers                                                     469263 non-null int64
civilians                                                    469263 non-null int64
county                                                       469263 non-null object
victim_sex_F                                                 469263 non-null uint8
victim_sex_M                                                 469263 non-null uint8
victim_sex_U                                                 469263 non-null uint8
crime_against_Not a Crime                                    469263 non-null uint8
crime_against_Person                                         469263 non-null uint8
crime_against_Property                                  

In [ ]:
#model_df.corr()

In [11]:
y = model_df.pop('county').values
#y = model_df.pop('fips').values
X = model_df.values

In [ ]:
# def standard_confusion_matrix(y_true, y_predict):
#     cm = confusion_matrix(y_true, y_predict)
#     cm = np.rot90(cm, k=1, axes=(0, 1))
#     cm = np.flip(cm, axis=1)
#     return cm

In [12]:
# Training and Testing Sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [13]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (351947, 79)
Training Labels Shape: (351947,)
Testing Features Shape: (117316, 79)
Testing Labels Shape: (117316,)


In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, auc, roc_curve, precision_recall_curve, precision_recall_fscore_support

/home/kendal/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [15]:
clf = RandomForestClassifier(oob_score=True, n_estimators=500, max_depth=50, max_features='auto')
clf.fit(X_train, y_train)
preds = clf.predict(X_test)
proba_preds = clf.predict_proba(X_test)

In [17]:
# calculate scores
precision_micro = precision_score(y_test, preds, average = 'micro')
precision_macro = precision_score(y_test, preds, average = 'macro')
precision_weighted = precision_score(y_test, preds, average = 'weighted')

recall_micro = recall_score(y_test, preds, average = 'micro')
recall_macro = recall_score(y_test, preds, average = 'macro')
recall_weighted = recall_score(y_test, preds, average = 'weighted')

F_1_micro = 2 / ((1/precision_micro) + (1/recall_micro))
F_1_macro = 2 / ((1/precision_macro) + (1/recall_macro))
F_1_weighted = 2 / ((1/precision_weighted) + (1/recall_weighted))

oob=clf.oob_score_
acc = accuracy_score(y_test, preds)

prfs_micro = precision_recall_fscore_support(y_test, preds, average='micro')
prfs_macro = precision_recall_fscore_support(y_test, preds, average='macro')
prfs_weighted = precision_recall_fscore_support(y_test, preds, average='weighted')

#Prec, Rec, Thresh = precision_recall_curve(y_test, proba_preds)

print('Precision Scores:', precision_micro, precision_macro, precision_weighted)
print('Recall Scores:', recall_micro, recall_macro, recall_weighted)
print('F 1 Scores:', F_1_micro, F_1_macro, F_1_weighted)
print('oob Score:', oob)
print('Accuracy Score:', acc)
print('PRF_Support Scores:', prfs_micro, prfs_macro, prfs_weighted)
#print(Prec, Rec, Thresh)

/home/kendal/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/kendal/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision Scores: 0.9926267516792253 0.875076758494342 0.9925242695678649
Recall Scores: 0.9926267516792253 0.7802483893299383 0.9926267516792253
F 1 Scores: 0.9926267516792253 0.8249463644682734 0.9925755079782594
oob Score: 0.9925358079483558
Accuracy Score: 0.9926267516792253
PRF_Support Scores: (0.9926267516792253, 0.9926267516792253, 0.9926267516792253, None) (0.875076758494342, 0.7802483893299383, 0.8093154922841049, None) (0.9925242695678649, 0.9926267516792253, 0.9923737565815585, None)


In [20]:
import operator
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(model_df.columns, clf.feature_importances_):
    feats[feature] = importance
feats = sorted(feats.items(), key = operator.itemgetter(1), reverse=True)
feats

[('population', 0.33320059434097155),
 ('officers', 0.3217283520308491),
 ('civilians', 0.30481838090222524),
 ('age_num', 0.0170427919336),
 ('offense_category_Fraud Offenses', 0.002173056921484733),
 ('location_id_18', 0.0015936489609905578),
 ('location_id_1', 0.0012528598568755846),
 ('location_id_20', 0.0012462014366213987),
 ('victim_sex_F', 0.0009396641630333506),
 ('victim_sex_M', 0.0009292935349417738),
 ('location_id_13', 0.0008590066640788812),
 ('location_id_25', 0.0008250081562365083),
 ('offense_category_Larceny/Theft Offenses', 0.0007711330794267947),
 ('location_id_41', 0.0006633516816633245),
 ('offense_category_Destruction/Damage/Vandalism of Property',
  0.0006610627328556825),
 ('offense_category_Burglary/Breaking & Entering', 0.000600934723346233),
 ('location_id_22', 0.0005483852492541915),
 ('offense_category_Kidnapping/Abduction', 0.000508610134979249),
 ('offense_category_Motor Vehicle Theft', 0.00044451619205791243),
 ('location_id_10', 0.0004443057788267813),